In [1]:
from rich import print

## Datasets

In the previous section we've seen that one of the main goals of springtime is
to harmonize datasets from different sources. Here, we walk through an example
with data from PEP725 and EOBS to show how this is done.


### PEP725

**Prerequisites: phenor**

To retrieve data from pep725, we make use of a pre-existing library called [phenor](https://bluegreen-labs.github.io/phenor/). Phenor is written in R, and therefore you need to have installed R with phenor. If you already have R on your system, you can install phenor like so:

```R
devtools::install_github("bluegreen-labs/phenor@v1.3.1")
```

If you don't have R, springtime provides a conda environment file that contains
most of the necessary dependencies, so you can create a conda environment like
so:

```sh
# Obtain the environment file
curl -o environment.yml https://raw.githubusercontent.com/phenology/springtime/main/environment.yml

# Create and activate the new environment
mamba env create --file environment.yml
conda activate springtime

# Install phenor in R
Rscript -e 'devtools::install_github("bluegreen-labs/phenor", upgrade="never")'
```

**PEP725 credentials**

To authenticate with the PEP725 data servers, you need to have an account and
you need to store your credentials in a file called
`~/.config/springtime/pep725_credentials.txt`. Email adress on first line,
password on second line. This path can be modified in the springtime
configuration, but the default is quite okay.


#### Springtime's dataset interface

Springtime provides a semi-standardized interface for working with datasets. In
this case, our we will be using `PEP725Phenor` as the dataset class. You can see the full documentation of this class [here](https://springtime.readthedocs.io/en/latest/reference/springtime/datasets/insitu/pep725/).

Let's create a dataset with all PEP725 observations of the species "Syringa vulgaris"


In [2]:
from springtime.datasets.insitu.pep725 import PEP725Phenor

dataset = PEP725Phenor(species="Syringa vulgaris", phenophase=11)
print(dataset)

PEP725Phenor(
    dataset='PEP725Phenor',
    years=None,
    credential_file=PosixPath('/home/peter/.config/springtime/pep725_credentials.txt'),
    species='Syringa vulgaris',
    phenophase=11,
    include_cols=['year', 'geometry', 'day'],
    area=None
)

Notice that the credential_file has been configured automatically, and that there are some other fields that we can set. Before we dive into details about what those options mean, we will need to retrieve the data. We can do this with the `download` method.


In [19]:
print(PEP725Phenor(species="Syringa vulgaris", years=[2000, 2002]).to_recipe())

dataset: PEP725Phenor
years:
- 2000
- 2002
species: Syringa vulgaris
include_cols:
- year
- geometry
- day

In [25]:
dataset = PEP725Phenor(species="Syringa vulgaris", years=[2000, 2002])
recipe = dataset.to_recipe()
assert recipe == dedent(
    """
    dataset: PEP725Phenor
    years:
    - 2000
    - 2002
    species: Syringa vulgaris
    include_cols:
    - year
    - geometry
    - day
    """
)

AssertionError: 

In [39]:
from textwrap import dedent

expected = dedent(
    """
    dataset: PEP725Phenor
    years:
    - 2000
    - 2002
    species: Syringa vulgaris
    include_cols:
    - year
    - geometry
    - day
    """
)
expected

'dataset: PEP725Phenor\n    years:\n    - 2000\n    - 2002\n    species: Syringa vulgaris\n    include_cols:\n    - year\n    - geometry\n    - day\n'

In [41]:
recipe

'dataset: PEP725Phenor\nyears:\n- 2000\n- 2002\nspecies: Syringa vulgaris\ninclude_cols:\n- year\n- geometry\n- day\n'

In [40]:
recipe == expected

False

In [33]:
recipe2 = PEP725Phenor(species="Syringa vulgaris", years=[2000, 2002]).to_recipe()

In [34]:
recipe == recipe2

True

In [3]:
dataset.download()

File already exists: /home/peter/.cache/springtime/PEP725/Syringa vulgaris.csv


If everything went well, the data should have been downloaded to some location
like `/home/username/.cache/springtime`. Springtime will skip the download if
the data is already present.

You can inspect the file on disk, but for transparancy springtime provides a
`raw_load` method that loads the data more or less without modification.


In [4]:
dataset.raw_load()

,pep_id,bbch,year,day,country,species,national_id,lon,lat,alt,name
0,6446,60,1991,130,AT,Syringa vulgaris,5120,14.4167,48.2167,225,ASTEN
1,6446,60,1984,137,AT,Syringa vulgaris,5120,14.4167,48.2167,225,ASTEN
2,6446,60,1969,124,AT,Syringa vulgaris,5120,14.4167,48.2167,225,ASTEN
3,6446,60,1989,107,AT,Syringa vulgaris,5120,14.4167,48.2167,225,ASTEN
4,6446,60,1990,112,AT,Syringa vulgaris,5120,14.4167,48.2167,225,ASTEN
...,...,...,...,...,...,...,...,...,...,...,...
173752,19283,60,2004,125,UK,Syringa vulgaris,964298,-3.7330,58.5670,-1,964298
173753,19283,60,2002,130,UK,Syringa vulgaris,964298,-3.7330,58.5670,-1,964298
173754,19283,60,2003,113,UK,Syringa vulgaris,964298,-3.7330,58.5670,-1,964298
173755,19285,60,2002,125,UK,Syringa vulgaris,968311,-3.5170,58.6000,-1,968311


As you can see, there are various columns in the data, only a few of which are relevant for us. The "day" column contains the day of year of the event. The event, in this case, is given in the 'bbch' column, which contains phenophases according to the BBCH scale. For example, phenophase 60 means "beginning of flowering". To see all possible options, have a look at http://www.pep725.eu/pep725_phase.php.

Note that this data is already interesting, but it doesn't completely conform to our standard yet. The `load` method, as opposed to `raw_load`, does some additional work to parse the data into a format that we can easily combine with other datasets.


In [5]:
dataset.load().reset_index(drop=True)

,year,geometry,day
0,1988,POINT (15.86660 44.80000),85
1,1981,POINT (15.86660 44.80000),83
2,1989,POINT (15.86660 44.80000),80
3,1985,POINT (15.86660 44.80000),94
4,2014,POINT (15.86660 44.80000),77
...,...,...,...
1426,2000,POINT (18.25000 49.11670),105
1427,2010,POINT (18.25000 49.11670),106
1428,2004,POINT (18.25000 49.11670),110
1429,2001,POINT (18.25000 49.11670),116


Notice that the year and geometry have been converted to index columns, we only
retained the "day" column, as this will be the variable that we are trying to
predict. The latitude and longitude have been combined into a "geometry" column
in geopandas format.

We can influence the behaviour of the `load` method to select an area and years of interest, for example. To this end, we need to modify the dataset.


In [6]:
area = {
    "name": "Germany",
    "bbox": [
        5.98865807458,
        47.3024876979,
        15.0169958839,
        54.983104153,
    ],
}
dataset = PEP725Phenor(species="Syringa vulgaris", years=[2000, 2002], area=area)
print(dataset)
df_pep725 = dataset.load()
df_pep725

PEP725Phenor(
    dataset='PEP725Phenor',
    years=YearRange(start=2000, end=2002),
    credential_file=PosixPath('/home/peter/.config/springtime/pep725_credentials.txt'),
    species='Syringa vulgaris',
    phenophase=None,
    include_cols=['year', 'geometry', 'day'],
    area=NamedArea(
        name='Germany',
        bbox=BoundingBox(xmin=5.98865807458, ymin=47.3024876979, xmax=15.0169958839, ymax=54.983104153)
    )
)

,year,geometry,day
0,2001,POINT (13.23330 47.78330),130
1,2000,POINT (13.23330 47.78330),131
2,2002,POINT (13.23330 47.78330),132
3,2002,POINT (14.88330 48.68330),122
4,2000,POINT (14.88330 48.68330),123
...,...,...,...
4718,2002,POINT (11.98330 50.70000),130
4719,2000,POINT (11.98330 50.70000),121
4720,2001,POINT (11.98330 50.70000),133
4721,2002,POINT (11.90000 50.65000),138


#### Dataset as recipe

You may wonder why we pass these additional arguments to the dataset itself; why not pass them directly to the load function? Part of the reason is standardization: most datasets need to know about the area and time already for downloading anything. By making it part of the dataset definition, datasets from several sources become more alike.

Another advantage of this model is that it allows us to export springtime datasets as "recipes".


In [7]:
recipe = dataset.to_recipe()
print(recipe)

dataset: PEP725Phenor
years:
- 2000
- 2002
species: Syringa vulgaris
include_cols:
- year
- geometry
- day
area:
  name: Germany
  bbox:
  - 5.98865807458
  - 47.3024876979
  - 15.0169958839
  - 54.983104153

These recipes are a `yaml` representation of the dataset definition. With their succinct and readible format, they can be stored and shared in a standardized way. We can easily load them again:


In [9]:
from springtime.datasets import load_dataset

reloaded_ds = load_dataset(recipe)
reloaded_ds == dataset

True

Moreover, springtime can read and execute these recipes from the command line as well. We will come back to this later, but the idea is that recipes can help to make data loading more reproducible and easier to automate.


## E-OBS

Now that we have observations (our target variables for the modelling part), we
need some predictor variables as well. Here, we will use e-obs.


For downloading e-obs we don't make use of an existing library. Instead, we simply download the data files directly from the [source](https://surfobs.climate.copernicus.eu/dataaccess/access_eobs.php).


In [10]:
from springtime.datasets.meteo.eobs import EOBS

ds_eobs = EOBS(
    years=["2000", "2002"],  # pyright: ignore (https://t.ly/gukmj)
    variables=[
        "mean_temperature",
        "minimum_temperature",
    ],
)
print(ds_eobs)
ds_eobs.download()

EOBS(
    dataset='E-OBS',
    years=YearRange(start=2000, end=2002),
    product_type='ensemble_mean',
    variables=['mean_temperature', 'minimum_temperature'],
    grid_resolution='0.1deg',
    version='26.0e'
)

/home/peter/.cache/springtime/e-obs/tg_ens_mean_0.1deg_reg_1995-2010_v26.0e.nc already exists, skipping
/home/peter/.cache/springtime/e-obs/tn_ens_mean_0.1deg_reg_1995-2010_v26.0e.nc already exists, skipping


The data comes in netCDF format, so we represent the raw data as an xarray object.


In [9]:
# TODO make this raw-load?
eobs_ds = ds_eobs.load()
eobs_ds

<xarray.Dataset>
Dimensions:              (latitude: 465, longitude: 705, time: 1096)
Coordinates:
  * latitude             (latitude) float64 25.05 25.15 25.25 ... 71.35 71.45
  * longitude            (longitude) float64 -24.95 -24.85 ... 45.35 45.45
  * time                 (time) datetime64[ns] 2000-01-01 ... 2002-12-31
Data variables:
    mean_temperature     (time, latitude, longitude) float32 dask.array<chunksize=(1096, 120, 183), meta=np.ndarray>
    minimum_temperature  (time, latitude, longitude) float32 dask.array<chunksize=(1096, 120, 183), meta=np.ndarray>
Attributes:
    E-OBS_version:  26.0e
    Conventions:    CF-1.4
    References:     http://surfobs.climate.copernicus.eu/dataaccess/access_eo...
    history:        Thu Sep 29 13:17:18 2022: ncks --no-abc -d time,16436,222...
    NCO:            netCDF Operators version 4.7.5 (Homepage = http://nco.sf....

Clearly, we need to do some more tweaking to reformat and extract the relevant
data, in order to match our standardized data format.

Firstly, notice that eobs has a time dimension that spans more than one record
per year, whereas our target data has only one unique row for each per
year/location. Thus, we need to reshape and/or aggregate the data.

Secondly, we need to extract only those points that are of interest. In this process, we choose the eobs grid cell that closest to the observations, recognizing that it might not be the exact same point. However, in order to join the datasets later on, we will use the input coordinates in the final dataframe.

**Dealing with time**

We start with the time dimension. While it is not impossible to work with daily data, for this example we are first going to resample it to monthly sums instead. Then, we'll split the time dimension in two: year and day of year.


In [10]:
# TODO: move to source code

import pandas as pd


def split_time(ds):
    """Split datetime coordinate into year and dayofyear."""
    year = ds.time.dt.year.values
    doy = ds.time.dt.dayofyear.values
    split_time = pd.MultiIndex.from_arrays(
        [year, doy],
        names=["year", "doy"],
    )
    return ds.assign_coords(time=split_time).unstack("time")

In [11]:
eobs_ds = eobs_ds.resample(time="M").sum()
eobs_ds = split_time(eobs_ds)
eobs_ds

<xarray.Dataset>
Dimensions:              (year: 3, doy: 23, latitude: 465, longitude: 705)
Coordinates:
  * year                 (year) int64 2000 2001 2002
  * doy                  (doy) int64 31 59 60 90 91 120 ... 305 334 335 365 366
  * latitude             (latitude) float64 25.05 25.15 25.25 ... 71.35 71.45
  * longitude            (longitude) float64 -24.95 -24.85 ... 45.35 45.45
Data variables:
    mean_temperature     (latitude, longitude, year, doy) float32 dask.array<chunksize=(120, 183, 1, 23), meta=np.ndarray>
    minimum_temperature  (latitude, longitude, year, doy) float32 dask.array<chunksize=(120, 183, 1, 23), meta=np.ndarray>
Attributes:
    E-OBS_version:  26.0e
    Conventions:    CF-1.4
    References:     http://surfobs.climate.copernicus.eu/dataaccess/access_eo...
    history:        Thu Sep 29 13:17:18 2022: ncks --no-abc -d time,16436,222...
    NCO:            netCDF Operators version 4.7.5 (Homepage = http://nco.sf....

**Extracing points / alignment with observations**

Next, we noted that e-obs is a gridded dataset, but we want to retrieve only those points for which
we have observations, so let's extract those. Two utility functions are available for this: extract points, or extract records. The difference is that extract records also takes the year index into account.


In [12]:
# TODO move to source code
# TODO reconcile with "pointsfromother"

import xarray as xr
import geopandas as gpd


def extract_points(ds, points: gpd.geodataframe.GeoDataFrame):
    """Extract list of points from gridded dataset."""
    x = xr.DataArray(points.unique().x, dims=["index"])
    y = xr.DataArray(points.unique().y, dims=["index"])
    return (
        ds.sel(longitude=x, latitude=y, method="nearest")
        .drop(["latitude", "longitude"])
        .assign(geometry=xr.DataArray(points.unique(), dims=["index"]))
    )


def extract_records(ds, records: gpd.geoseries.GeoSeries):
    """Extract list of year/geometry records from gridded dataset."""
    x = records.geometry.x.to_xarray()
    y = records.geometry.y.to_xarray()
    year = records.year.to_xarray()
    geometry = xr.DataArray(records.geometry, dims=["index"])
    # TODO ensure all years present before allowing 'nearest' on year
    return (
        ds.sel(longitude=x, latitude=y, year=year, method="nearest")
        .drop(["latitude", "longitude"])
        .assign(year=year, geometry=geometry)
    )

In [13]:
eobs_ds = extract_records(eobs_ds, df_pep725)
eobs_ds

<xarray.Dataset>
Dimensions:              (index: 4723, doy: 23)
Coordinates:
    year                 (index) int64 2001 2000 2002 2002 ... 2001 2002 2000
  * doy                  (doy) int64 31 59 60 90 91 120 ... 305 334 335 365 366
  * index                (index) int64 0 1 2 3 4 5 ... 4718 4719 4720 4721 4722
Data variables:
    mean_temperature     (index, doy) float32 dask.array<chunksize=(4723, 23), meta=np.ndarray>
    minimum_temperature  (index, doy) float32 dask.array<chunksize=(4723, 23), meta=np.ndarray>
    geometry             (index) object POINT (13.2333 47.7833) ... POINT (11...
Attributes:
    E-OBS_version:  26.0e
    Conventions:    CF-1.4
    References:     http://surfobs.climate.copernicus.eu/dataaccess/access_eo...
    history:        Thu Sep 29 13:17:18 2022: ncks --no-abc -d time,16436,222...
    NCO:            netCDF Operators version 4.7.5 (Homepage = http://nco.sf....

At this stage, most of the heavy lifting is done, and the size of the total dataset is substantially reduced. Now, we can convert our data to a dataframe.


In [14]:
eobs_df = eobs_ds.to_dataframe()
eobs_df

year  mean_temperature  minimum_temperature  \
index doy                                                
0     31   2001         -0.860000          -103.929985   
      59   2001         32.549995           -54.980000   
      60   2001               NaN                  NaN   
      90   2001        165.709976            44.709999   
      91   2001               NaN                  NaN   
...         ...               ...                  ...   
4722  305  2000        324.219971           236.010010   
      334  2000               NaN                  NaN   
      335  2000        168.010010            83.489990   
      365  2000               NaN                  NaN   
      366  2000         75.809998             3.960004   

                          geometry  
index doy                           
0     31   POINT (13.2333 47.7833)  
      59   POINT (13.2333 47.7833)  
      60   POINT (13.2333 47.7833)  
      90   POINT (13.2333 47.7833)  
      91   POINT (13.2333 47.7833)  
...                            ...  
4722  305       POINT (11.9 50.65)  
      334       POINT (11.9 50.65)  
      335       POINT (11.9 50.65)  
      365       POINT (11.9 50.65)  
      366       POINT (11.9 50.65)  

[108629 rows x 4 columns]

Notice that the DOY is still an index column. Since we want only one record per location/year, we can stack the DOY column and combine it with the variable name. Effectively, it means we treat the cumulative temperature for each month as a separate predictor.


In [15]:
eobs_df = eobs_df.set_index(["year", "geometry"], append=True).unstack("doy")
eobs_df.columns = eobs_df.columns.map("{0[0]}|{0[1]}".format)
eobs_df = eobs_df.reset_index("index", drop=True).reset_index()
eobs_df = gpd.GeoDataFrame(eobs_df)
eobs_df

,year,geometry,mean_temperature|31,mean_temperature|59,mean_temperature|60,mean_temperature|90,mean_temperature|91,mean_temperature|120,mean_temperature|121,mean_temperature|151,...,minimum_temperature|243,minimum_temperature|244,minimum_temperature|273,minimum_temperature|274,minimum_temperature|304,minimum_temperature|305,minimum_temperature|334,minimum_temperature|335,minimum_temperature|365,minimum_temperature|366
0,2001,POINT (13.23330 47.78330),-0.860000,32.549995,NaN,165.709976,NaN,158.689987,NaN,441.069946,...,416.730072,NaN,199.339966,NaN,288.010010,NaN,-45.559998,NaN,-206.860001,NaN
1,2000,POINT (13.23330 47.78330),-74.610001,NaN,51.149994,NaN,88.860001,NaN,288.369965,NaN,...,NaN,417.289978,NaN,286.439972,NaN,213.929993,NaN,56.709995,NaN,-0.759994
2,2002,POINT (13.23330 47.78330),-23.209991,103.779991,NaN,150.759979,NaN,193.299988,NaN,420.369965,...,398.309937,NaN,229.170029,NaN,146.220001,NaN,81.270004,NaN,-44.820000,NaN
3,2002,POINT (14.88330 48.68330),-67.640007,85.110001,NaN,116.720009,NaN,197.449997,NaN,434.310059,...,372.389984,NaN,183.919983,NaN,84.409996,NaN,18.409998,NaN,-135.939987,NaN
4,2000,POINT (14.88330 48.68330),-131.679993,NaN,49.069996,NaN,80.399994,NaN,287.309998,NaN,...,NaN,332.979980,NaN,213.639984,NaN,179.450012,NaN,-1.489999,NaN,-102.730003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4718,2002,POINT (11.98330 50.70000),14.540002,127.029991,NaN,145.490005,NaN,206.809998,NaN,419.690033,...,449.339996,NaN,254.349976,NaN,142.500000,NaN,74.690002,NaN,-104.629997,NaN
4719,2000,POINT (11.98330 50.70000),6.549999,NaN,103.329994,NaN,144.540009,NaN,300.600006,NaN,...,NaN,382.760040,NaN,302.750000,NaN,242.170013,NaN,87.580002,NaN,13.430005
4720,2001,POINT (11.98330 50.70000),1.390004,56.449993,NaN,113.529991,NaN,205.859985,NaN,422.839966,...,420.369995,NaN,259.820007,NaN,265.889984,NaN,20.139999,NaN,-122.150002,NaN
4721,2002,POINT (11.90000 50.65000),3.379994,115.690002,NaN,135.319977,NaN,196.209991,NaN,410.709991,...,437.399963,NaN,242.609955,NaN,134.829987,NaN,67.330002,NaN,-110.429993,NaN


Finally, our e-obs data have the exact same format as the PEP725 observations. The next step will be to merge the dataframes together.


In [16]:
def join_dataframes(dfs, index_cols=["year", "geometry"]):
    """Join dataframes by index cols.

    Assumes incoming data is a geopandas dataframe with a geometry column. Not
    as index.
    """
    others = []
    for df in dfs:
        df = gpd.GeoDataFrame(df)  # TODO should not be necessary
        df = df.to_wkt()
        df.set_index(index_cols, inplace=True)
        others.append(df)

    main_df = others.pop(0)

    df = main_df.join(others, how="outer")
    df.reset_index(inplace=True)
    geometry = gpd.GeoSeries.from_wkt(df.pop("geometry"))

    return gpd.GeoDataFrame(df, geometry=geometry).set_index(index_cols)


join_dataframes([df_pep725, eobs_df])

day  mean_temperature|31  mean_temperature|59  \
year geometry                                                                   
2000 POINT (10.00000 49.48330)  129            10.029995                  NaN   
     POINT (10.00000 50.85000)  120            29.239996                  NaN   
     POINT (10.00000 51.71670)  116            52.520008                  NaN   
     POINT (10.00000 52.10000)  120            78.489983                  NaN   
     POINT (10.00000 53.08330)  121            65.709991                  NaN   
...                             ...                  ...                  ...   
2002 POINT (9.96667 50.15000)   120             1.049998           143.389984   
     POINT (9.96667 50.95000)   131            20.680006           141.419998   
     POINT (9.96667 52.81670)   131            78.469994           133.709991   
     POINT (9.98333 49.76670)   118             6.870003           162.500015   
     POINT (9.98333 53.36670)   127            99.659996           149.600006   

                                mean_temperature|60  mean_temperature|90  \
year geometry                                                              
2000 POINT (10.00000 49.48330)           106.269997                  NaN   
     POINT (10.00000 50.85000)           110.070007                  NaN   
     POINT (10.00000 51.71670)           117.549995                  NaN   
     POINT (10.00000 52.10000)           143.180008                  NaN   
     POINT (10.00000 53.08330)           115.659996                  NaN   
...                                             ...                  ...   
2002 POINT (9.96667 50.15000)                   NaN           179.099991   
     POINT (9.96667 50.95000)                   NaN           147.550003   
     POINT (9.96667 52.81670)                   NaN           149.360001   
     POINT (9.98333 49.76670)                   NaN           198.879990   
     POINT (9.98333 53.36670)                   NaN           155.910019   

                                mean_temperature|91  mean_temperature|120  \
year geometry                                                               
2000 POINT (10.00000 49.48330)           166.119995                   NaN   
     POINT (10.00000 50.85000)           175.479996                   NaN   
     POINT (10.00000 51.71670)           167.379990                   NaN   
     POINT (10.00000 52.10000)           178.909973                   NaN   
     POINT (10.00000 53.08330)           149.180008                   NaN   
...                                             ...                   ...   
2002 POINT (9.96667 50.15000)                   NaN            261.339966   
     POINT (9.96667 50.95000)                   NaN            226.040009   
     POINT (9.96667 52.81670)                   NaN            232.060013   
     POINT (9.98333 49.76670)                   NaN            280.159973   
     POINT (9.98333 53.36670)                   NaN            242.510010   

                                mean_temperature|121  mean_temperature|151  \
year geometry                                                                
2000 POINT (10.00000 49.48330)            299.009979                   NaN   
     POINT (10.00000 50.85000)            300.040070                   NaN   
     POINT (10.00000 51.71670)            316.889984                   NaN   
     POINT (10.00000 52.10000)            329.799988                   NaN   
     POINT (10.00000 53.08330)            297.209961                   NaN   
...                                              ...                   ...   
2002 POINT (9.96667 50.15000)                    NaN            423.109894   
     POINT (9.96667 50.95000)                    NaN            407.989960   
     POINT (9.96667 52.81670)                    NaN            435.720001   
     POINT (9.98333 49.76670)                    NaN            439.520020   
     POINT (9.98333 53.36670)                    Na

### Summary

Bringing everything together, we can reduce this whole notebook to a few lines of code.


In [17]:
from springtime.datasets.meteo.eobs import EOBS

area = {
    "name": "Germany",
    "bbox": [
        5.98865807458,
        47.3024876979,
        15.0169958839,
        54.983104153,
    ],
}

df_pep725 = PEP725Phenor(
    species="Syringa vulgaris", years=[2000, 2002], area=area
).load()

# TODO make this work
df_eobs = EOBS(
    years=["2000", "2002"],
    variables=["mean_temperature", "minimum_temperature"],
    resample="M",
    operator="sum",
    points=df_pep725.geometry,  # TODO: use pointsfom other here?
).load()

join_dataframes([df_pep725, eobs_df])

# TODO export to recipe

day  mean_temperature|31  mean_temperature|59  \
year geometry                                                                   
2000 POINT (10.00000 49.48330)  129            10.029995                  NaN   
     POINT (10.00000 50.85000)  120            29.239996                  NaN   
     POINT (10.00000 51.71670)  116            52.520008                  NaN   
     POINT (10.00000 52.10000)  120            78.489983                  NaN   
     POINT (10.00000 53.08330)  121            65.709991                  NaN   
...                             ...                  ...                  ...   
2002 POINT (9.96667 50.15000)   120             1.049998           143.389984   
     POINT (9.96667 50.95000)   131            20.680006           141.419998   
     POINT (9.96667 52.81670)   131            78.469994           133.709991   
     POINT (9.98333 49.76670)   118             6.870003           162.500015   
     POINT (9.98333 53.36670)   127            99.659996           149.600006   

                                mean_temperature|60  mean_temperature|90  \
year geometry                                                              
2000 POINT (10.00000 49.48330)           106.269997                  NaN   
     POINT (10.00000 50.85000)           110.070007                  NaN   
     POINT (10.00000 51.71670)           117.549995                  NaN   
     POINT (10.00000 52.10000)           143.180008                  NaN   
     POINT (10.00000 53.08330)           115.659996                  NaN   
...                                             ...                  ...   
2002 POINT (9.96667 50.15000)                   NaN           179.099991   
     POINT (9.96667 50.95000)                   NaN           147.550003   
     POINT (9.96667 52.81670)                   NaN           149.360001   
     POINT (9.98333 49.76670)                   NaN           198.879990   
     POINT (9.98333 53.36670)                   NaN           155.910019   

                                mean_temperature|91  mean_temperature|120  \
year geometry                                                               
2000 POINT (10.00000 49.48330)           166.119995                   NaN   
     POINT (10.00000 50.85000)           175.479996                   NaN   
     POINT (10.00000 51.71670)           167.379990                   NaN   
     POINT (10.00000 52.10000)           178.909973                   NaN   
     POINT (10.00000 53.08330)           149.180008                   NaN   
...                                             ...                   ...   
2002 POINT (9.96667 50.15000)                   NaN            261.339966   
     POINT (9.96667 50.95000)                   NaN            226.040009   
     POINT (9.96667 52.81670)                   NaN            232.060013   
     POINT (9.98333 49.76670)                   NaN            280.159973   
     POINT (9.98333 53.36670)                   NaN            242.510010   

                                mean_temperature|121  mean_temperature|151  \
year geometry                                                                
2000 POINT (10.00000 49.48330)            299.009979                   NaN   
     POINT (10.00000 50.85000)            300.040070                   NaN   
     POINT (10.00000 51.71670)            316.889984                   NaN   
     POINT (10.00000 52.10000)            329.799988                   NaN   
     POINT (10.00000 53.08330)            297.209961                   NaN   
...                                              ...                   ...   
2002 POINT (9.96667 50.15000)                    NaN            423.109894   
     POINT (9.96667 50.95000)                    NaN            407.989960   
     POINT (9.96667 52.81670)                    NaN            435.720001   
     POINT (9.98333 49.76670)                    NaN            439.520020   
     POINT (9.98333 53.36670)                    Na

# To Do:

- EOBS load function doesn't work (yet)
- Make all eobs classes into one?
- Make sure resample works on EOBS
- Make sure we can (de)serialize individual datasets: `ds = dataset.from_yaml(dataset.to_yaml())`
- Make sure recipes work
- Move snippets to source code
- Can we pass primitive types to pydantic?
